In [1]:
import pathlib
from datetime import datetime
from typing import List, Tuple, Union, Dict

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Column

from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
# master configuration to use only 4 CPU cores
spark = SparkSession.builder.master("local[4]").getOrCreate()

# basic configuration to use only a reasonable number of partitions
spark.conf.set("spark.sql.shuffle.partition", 4)

# configuration to work in UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/07 12:41:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = "/Users/emilianofrigo/Downloads/test_read/"

# Read JSONL file
df = (
    spark.read
    # .option("multiline", "true")
    .json(path)
)

23/08/07 12:41:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df.printSchema()

root
 |-- _airbyte_ab_id: string (nullable = true)
 |-- _airbyte_data: struct (nullable = true)
 |    |-- AdjustmentEventList: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- AdjustmentAmount: struct (nullable = true)
 |    |    |    |    |-- CurrencyAmount: double (nullable = true)
 |    |    |    |    |-- CurrencyCode: string (nullable = true)
 |    |    |    |-- AdjustmentItemList: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- PerUnitAmount: struct (nullable = true)
 |    |    |    |    |    |    |-- CurrencyAmount: double (nullable = true)
 |    |    |    |    |    |    |-- CurrencyCode: string (nullable = true)
 |    |    |    |    |    |-- ProductDescription: string (nullable = true)
 |    |    |    |    |    |-- Quantity: string (nullable = true)
 |    |    |    |    |    |-- SellerSKU: string (nullable = true)
 |    |    |    |    |    |-- TotalAmount: struct 

In [15]:
df.count()

809

23/08/07 18:12:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2083846 ms exceeds timeout 120000 ms
23/08/07 18:12:40 WARN SparkContext: Killing executors is not supported by current scheduler.


In [5]:
df.show(truncate=False)

+------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
def explode(df: DataFrame, config: dict) -> DataFrame:
    columns = config["columns"]
    if not isinstance(columns, list):
        columns = [columns]
    for column in columns:
        df.withColumn(column, F.explode(column))
        print(f"    {column} count: {df.count()}")
    return df

In [8]:
def flatten(input_sdf: DataFrame):
    event_sdf = input_sdf.withColumn(
        "AmazonOrderId", F.explode("_airbyte_data.ServiceFeeEventList.AmazonOrderId")
    )

    return event_sdf

In [9]:
df_test = flatten(df)

In [10]:
df_test.printSchema()
df_test.show()

root
 |-- _airbyte_ab_id: string (nullable = true)
 |-- _airbyte_data: struct (nullable = true)
 |    |-- AdjustmentEventList: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- AdjustmentAmount: struct (nullable = true)
 |    |    |    |    |-- CurrencyAmount: double (nullable = true)
 |    |    |    |    |-- CurrencyCode: string (nullable = true)
 |    |    |    |-- AdjustmentItemList: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- PerUnitAmount: struct (nullable = true)
 |    |    |    |    |    |    |-- CurrencyAmount: double (nullable = true)
 |    |    |    |    |    |    |-- CurrencyCode: string (nullable = true)
 |    |    |    |    |    |-- ProductDescription: string (nullable = true)
 |    |    |    |    |    |-- Quantity: string (nullable = true)
 |    |    |    |    |    |-- SellerSKU: string (nullable = true)
 |    |    |    |    |    |-- TotalAmount: struct 

+--------------------+--------------------+-------------------+-------------------+
|      _airbyte_ab_id|       _airbyte_data|_airbyte_emitted_at|      AmazonOrderId|
+--------------------+--------------------+-------------------+-------------------+
|63d4c2f1-d257-465...|{[], [], [], [], ...|      1691033334643|               null|
|e9df3674-4d53-4d3...|{[{{2.41, GBP}, [...|      1691033336076|               null|
|e9df3674-4d53-4d3...|{[{{2.41, GBP}, [...|      1691033336076|               null|
|a78dc35a-0c63-483...|{[], [], [], [], ...|      1691033343522|407-6935280-8029123|
|a78dc35a-0c63-483...|{[], [], [], [], ...|      1691033343522|407-6935280-8029123|
|d4395b9b-cf39-476...|{[], [], [], [], ...|      1691033344033|               null|
|d4395b9b-cf39-476...|{[], [], [], [], ...|      1691033344033|               null|
|d4395b9b-cf39-476...|{[], [], [], [], ...|      1691033344033|               null|
|d4395b9b-cf39-476...|{[], [], [], [], ...|      1691033344033|             

In [14]:
df_test.count()

472